In [ ]:
# !pip install transformers[torch] datasets evaluate

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
from transformers import pipeline
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
classifier = pipeline("sentiment-analysis")

In [ ]:
classifier("The actors were very convincing")

In [ ]:
classifier.model

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

input = tokenizer("The actors were very convincing", return_tensors="pt")
model_output = model(**input, output_hidden_states=True)
model_output["hidden_states"][-1]

In [ ]:
model = AutoModel.from_pretrained("google-t5/t5-base")
model

In [ ]:
model = AutoModel.from_pretrained("google-bert/bert-base-uncased")
model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
input = tokenizer("The actors were very convincing", return_tensors="pt")
model_output = model(**input)
torch.mean(model_output[0], 1).size()

torch.Size([1, 768])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ModelForTextClassification(nn.Module):
  def __init__(self, num_labels=2):
    super().__init__()
    self.bertEncoder = AutoModel.from_pretrained("google-bert/bert-base-uncased")
    self.dropout = nn.Dropout(0.2)
    self.classification = nn.Linear(768, num_labels)

  def forward(self, x):
    bertOutput = self.bertEncoder(x)
    # out = self.dropout(bertOutput[0][:, 0, :])
    out = self.dropout(torch.mean(bertOutput[0], 1))
    out = self.classification(out)
    return out


In [ ]:
model = ModelForTextClassification()
model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=3)
model

In [ ]:
model_output["last_hidden_state"]

In [ ]:
model_output[0]

In [ ]:
imdb_dataset = load_dataset("imdb")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb_dataset.map(preprocess_function, batched=True)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.205300,0.183544,0.930760


Epoch,Training Loss,Validation Loss,Accuracy
1,0.205300,0.183544,0.930760
2,0.120300,0.218962,0.941120


TrainOutput(global_step=3126, training_loss=0.19046622976155442, metrics={'train_runtime': 6341.3497, 'train_samples_per_second': 7.885, 'train_steps_per_second': 0.493, 'total_flos': 1.302353679597312e+16, 'train_loss': 0.19046622976155442, 'epoch': 2.0})

**Inference**

In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [ ]:
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
import torch

with torch.no_grad():
    logits = model.to("cpu")(**inputs).logits

In [ ]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'POSITIVE'